In [1]:
import os
import random
import time
import numpy as np
import json
import logging
import argparse
import torch
import torch.backends.cudnn as cudnn
from torch.nn.functional import logsigmoid
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
import torch.utils.data
import torch.multiprocessing as mp
import torch.distributed as dist
# from torch.utils.tensorboard import SummaryWriter

import csv
from torch.optim import Adam
from sys import argv
import json
import pdb
from torch.nn import *
from collections import defaultdict
from tqdm import tqdm
import pandas as pd
from datetime import datetime
import shutil
import yaml

from data.utility import Dataset
from trainer.TransMatch_pretrain import TransMatch
from trainer.TransE import TransE
from util.eval_utils import *

In [2]:
train_df = pd.read_csv("data/iqon_s/train.csv", header=None).astype('int')
train_df.columns=["user_idx", "top_idx", "pos_bottom_idx", "neg_bottom_idx"]
test_df = pd.read_csv("data/iqon_s/test.csv", header=None).astype('int')
test_df.columns=["user_idx", "top_idx", "pos_bottom_idx", "neg_bottom_idx"]
valid_df = pd.read_csv("data/iqon_s/val.csv", header=None).astype('int')
valid_df.columns=["user_idx", "top_idx", "pos_bottom_idx", "neg_bottom_idx"]
all_bottoms_id = pd.concat([train_df["pos_bottom_idx"], test_df["pos_bottom_idx"], valid_df["pos_bottom_idx"],
    train_df["neg_bottom_idx"], test_df["neg_bottom_idx"], valid_df["neg_bottom_idx"]], ignore_index=True).unique()

In [10]:
all_user = pd.concat([train_df["user_idx"], test_df["user_idx"], valid_df["user_idx"]], ignore_index=True).unique()
all_user

array([1506, 1598, 1748, ...,  350, 1261,  772])

In [11]:
len(all_user)

1769

In [3]:
train_ij_pairs = train_df[['top_idx', 'pos_bottom_idx']].drop_duplicates().values.tolist()
train_ij_pairs

[[33369, 55036],
 [32593, 16447],
 [15739, 58334],
 [22463, 11177],
 [9183, 86612],
 [57917, 48237],
 [45135, 13483],
 [39982, 40063],
 [72132, 28499],
 [66205, 25554],
 [7442, 81704],
 [46207, 47145],
 [85516, 18247],
 [74925, 62290],
 [45707, 5728],
 [67239, 4779],
 [78981, 31783],
 [53465, 21087],
 [66282, 62507],
 [15366, 647],
 [71121, 6021],
 [87273, 49570],
 [31679, 62023],
 [31883, 73614],
 [51870, 15916],
 [83971, 40068],
 [18712, 63934],
 [41210, 66873],
 [37886, 29255],
 [78986, 88701],
 [34271, 91698],
 [41686, 38406],
 [16917, 46096],
 [49838, 90556],
 [81906, 54949],
 [40610, 75525],
 [71732, 55515],
 [85275, 61313],
 [63582, 33562],
 [61202, 8413],
 [19221, 61268],
 [10007, 2661],
 [90509, 89790],
 [6308, 69793],
 [52367, 92910],
 [67071, 85598],
 [10568, 16054],
 [28646, 30315],
 [34271, 34533],
 [13672, 82366],
 [11807, 42654],
 [4593, 9062],
 [85323, 39669],
 [34687, 24743],
 [46876, 31619],
 [11232, 43569],
 [20743, 65981],
 [26625, 71003],
 [37093, 65555],
 [71082, 

In [16]:
a=0
new_u_ij_dict = {}
dataset.visual_features = dataset.visual_features.to(conf['device'])

ij_pairs = to_tensor(train_ij_pairs)
Is = ij_pairs[:, 0]
Js = ij_pairs[:, 1]
i_rep = model.i_embeddings_i(Is)
j_rep = model.i_embeddings_i(Is)
j_bias = model.i_bias_l(Js)
vis_I = dataset.visual_features[Is]
vis_J = dataset.visual_features[Js]
I_visual = model.visual_nn_comp(vis_I) #bs, hidden_dim
J_visual = model.visual_nn_comp(vis_J)
J_bias_v = model.i_bias_v(Js)

for user_idx in all_user:  
    u_idx = to_tensor(user_idx)   #key
    u_rep = model.u_embeddings_l(u_idx.expand(Is.size(0))) #Is.size(0), hd
    distances = model.transE_predict(u_rep, i_rep, j_rep, j_bias)
    u_rep_v = model.u_embeddings_v(u_idx.expand(Is.size(0))) #Is.size(0), hd
    distances_v = model.transE_predict(u_rep, I_visual, J_visual, J_bias_v)
    distances += distances_v

    topk_scores, topk_indices = torch.topk(distances.view(-1), conf['top_k_u'], dim=-1)
    topk_i_j_pairs = ij_pairs[topk_indices]
    new_u_ij_dict[int(user_idx)] = topk_i_j_pairs.cpu().numpy().tolist()
    if a < 1:
        print(new_u_ij_dict)
        a+= 1
    else:
        continue
with open('data/iqon_s/u_topk_ijs_dict.json', 'w') as json_file:
    json.dump(new_u_ij_dict, json_file)  

{1506: [[22073, 62023], [26741, 62023], [11669, 62023], [13686, 62023], [45560, 62023]]}


In [18]:
u_ij_dict = train_df.groupby('user_idx').apply(lambda x: x[['top_idx', 'pos_bottom_idx']].values.tolist()).to_dict()
u_ij_dict

{0: [[18188, 38352],
  [82533, 58038],
  [19133, 93990],
  [34476, 65028],
  [28604, 14755],
  [62718, 51008],
  [44388, 72081],
  [11576, 30864],
  [31210, 60070],
  [19576, 60053],
  [25445, 83751],
  [61961, 18657],
  [59659, 39004],
  [11373, 48968],
  [77751, 69611],
  [87169, 31023],
  [13440, 74214],
  [47394, 13135],
  [9500, 811],
  [42038, 59621],
  [35359, 70151],
  [58782, 49660],
  [11301, 90608],
  [57744, 28585]],
 1: [[71017, 17674],
  [19611, 11896],
  [38491, 10678],
  [92412, 77904],
  [20795, 82690],
  [41964, 83127],
  [63780, 12309],
  [38607, 49543],
  [86146, 36816]],
 2: [[76835, 13104],
  [79170, 43443],
  [3078, 29567],
  [11722, 65396],
  [53222, 24666],
  [49827, 23245],
  [79170, 46381],
  [41248, 58175],
  [81209, 42511],
  [60058, 40422],
  [10760, 13104],
  [42416, 41428],
  [47745, 56768],
  [20542, 13104],
  [74420, 40980],
  [34906, 487],
  [13104, 26759],
  [11543, 69173],
  [4158, 34906]],
 3: [[34445, 48431],
  [22740, 86214],
  [53530, 20046],
  

In [5]:
def get_logger():
    logger_name = "main-logger"
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.INFO)
    handler = logging.StreamHandler()
    fmt = "[%(asctime)s %(levelname)s %(filename)s line %(lineno)d %(process)d] %(message)s"
    handler.setFormatter(logging.Formatter(fmt))
    logger.addHandler(handler)
    return logger
conf = yaml.safe_load(open("./config/train_model_config.yaml"))
conf["dataset"] = "iqon_s"
conf["gpu"] = 1
conf["device"] = torch.device("cuda:%s"%conf["gpu"] if torch.cuda.is_available() else "cpu")
dataset = Dataset(conf)
global logger
logger = get_logger()

In [6]:
conf["user_num"] = len(dataset.user_map)
conf["item_num"] = len(dataset.item_map)
conf["cate_num"] = len(dataset.cate_items)
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
conf['pretrained_model'] = 'TransE'
pretrain_model_file = f"{conf['pretrained_model']}.pth.tar"
pretrain_model_dir = "model/iqon_s/pretrained_model/"
pretrain_model_path = os.path.join(pretrain_model_dir, pretrain_model_file)
if os.path.exists(pretrain_model_path):
    logger.info("=> loading model ...")
    model = torch.load(pretrain_model_path)
    print("Testing with existing model...")
    conf['use_pretrain'] = True
    model.to(conf["device"])
    logger.info(model)

[2024-01-10 17:21:17,020 INFO 2687879523.py line 10 3025312] => loading model ...


Testing with existing model...


[2024-01-10 17:21:19,579 INFO 2687879523.py line 15 3025312] TransE(
  (u_embeddings_l): Embedding(1770, 32, padding_idx=1769)
  (i_bias_l): Embedding(94250, 1, padding_idx=94249)
  (i_embeddings_i): Embedding(94250, 32, padding_idx=94249)
  (visual_nn_comp): Sequential(
    (0): Linear(in_features=2048, out_features=32, bias=True)
    (1): Sigmoid()
  )
  (visual_nn_per): Sequential(
    (0): Linear(in_features=2048, out_features=32, bias=True)
    (1): Sigmoid()
  )
  (i_bias_v): Embedding(94250, 1, padding_idx=94249)
  (u_embeddings_v): Embedding(1770, 32, padding_idx=1769)
)


In [7]:
def to_tensor(data):
    return torch.tensor(data, dtype=torch.int64).to(conf["device"])

In [26]:
u_ij_dict[0][:1]

[[18188, 38352]]

In [54]:
a=0
for user_idx, i_j_pairs in u_ij_dict.items(): # key, value
    i = to_tensor([item[0] for item in i_j_pairs])  # 获取所有 'i' 的值
    j = to_tensor([item[1] for item in i_j_pairs])
    u = to_tensor(user_idx)  
    if a < 1:
        print(i.size())
        print(u.expand(i.size(0)))
        print(i)
        print(j)
        a+= 1

    else:
        break

torch.Size([24])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:1')
tensor([18188, 82533, 19133, 34476, 28604, 62718, 44388, 11576, 31210, 19576,
        25445, 61961, 59659, 11373, 77751, 87169, 13440, 47394,  9500, 42038,
        35359, 58782, 11301, 57744], device='cuda:1')
tensor([38352, 58038, 93990, 65028, 14755, 51008, 72081, 30864, 60070, 60053,
        83751, 18657, 39004, 48968, 69611, 31023, 74214, 13135,   811, 59621,
        70151, 49660, 90608, 28585], device='cuda:1')


In [21]:
a=0
new_u_ij_dict = {}
for user_idx, i_j_pairs in u_ij_dict.items(): # key, value
    Is = to_tensor([item[0] for item in i_j_pairs])  # 获取所有 'i' 的值
    Js = to_tensor([item[1] for item in i_j_pairs]) 
    ijs = to_tensor(i_j_pairs)
    u_idx = to_tensor(user_idx)   #key

    u_rep = model.u_embeddings_l(u_idx.expand(Is.size(0))) #Is.size(0), hd
    i_rep = model.i_embeddings_i(Is)
    j_rep = model.i_embeddings_i(Is)
    j_bias = model.i_bias_l(Js)

    distances = model.transE_predict(u_rep, i_rep, j_rep, j_bias)

    u_rep_v = model.u_embeddings_v(u_idx.expand(Is.size(0))) #Is.size(0), hd
    vis_I = dataset.visual_features[Is]
    vis_J = dataset.visual_features[Js]
    I_visual = model.visual_nn_comp(vis_I) #bs, hidden_dim
    J_visual = model.visual_nn_comp(vis_J)
    J_bias_v = model.i_bias_v(Js)

    distances_v = model.transE_predict(u_rep, I_visual, J_visual, J_bias_v)
    distances += distances_v

    topk_scores, topk_indices = torch.topk(distances.view(-1), conf['top_k_u'], dim=-1)
    topk_i_j_pairs = ijs[topk_indices]
    new_u_ij_dict[int(user_idx)] = topk_i_j_pairs.cpu().numpy().tolist()
    if a < 1:
        print(new_u_ij_dict)
        a+= 1
    else:
        continue
with open('data/iqon_s/u_exist_topk_ijs_dict.json', 'w') as json_file:
    json.dump(new_u_ij_dict, json_file)  

{0: [[47394, 13135], [28604, 14755], [19133, 93990], [87169, 31023], [25445, 83751]]}
